# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, ParametricMembrane, Volume, \
                        Cell, Sugars, Vector2, Vector3, Quaternion
import nglview

be = BioExplorer('localhost:5000')

In [ ]:
status = be.reset()

### Resources

In [ ]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
membrane_folder = pdb_folder + 'membrane/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

### Configuration

In [ ]:
# Scene
scene_size = Vector3(250.0, 250.0, 250.0)

# Immune system
nb_glucoses = 3600
nb_lactoferrins = 10
nb_defensins = 30

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

## Camera

In [ ]:
status = be.core_api().set_camera(
    orientation=[0.0, 0.0, 0.0, 1.0],
    position=[4.883, 44.255, 431.911],
    target=[4.883, 44.255, 31.311]
)

## Scene description

### Coronavirus

In [ ]:
name='Coronavirus'
be.add_coronavirus(
    name=name, resource_folder=resource_folder, 
    representation=protein_representation, 
    atom_radius_multiplier=protein_radius_multiplier,
    add_glycans=True, position=Vector3(75.0, 0.0, 60.0)
)

### Host cell

In [ ]:
# ACE2 receptor definition
ace2_receptor = Protein(
    sources=[pdb_folder + '6m18.pdb'],
    occurences=1,
    position=Vector3(0.0, 6.0, 0.0))

# Membrane definition
membrane_size = scene_size
membrane_height = 80.0
membrane = ParametricMembrane(
    sources=[
        membrane_folder + 'segA.pdb',
        membrane_folder + 'segB.pdb',
        membrane_folder + 'segC.pdb',
        membrane_folder + 'segD.pdb'
    ],
    occurences=30000)

# Cell definition
name='Cell'
cell = Cell(
    name=name, size=Vector2(membrane_size.x, membrane_size.z), shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    membrane=membrane, receptor=ace2_receptor, extra_parameters=[membrane_height])

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=Vector3(0.0, -80.0, 0.0),
    random_seed=1
)

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)

In [ ]:
be.get_protein_amino_acid_information(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR)

In [ ]:
glycan_folder = pdb_folder + 'glycans/'
complex_paths = [glycan_folder + 'complex/5.pdb', glycan_folder + 'complex/15.pdb',
                 glycan_folder + 'complex/25.pdb',glycan_folder + 'complex/35.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/20.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/1.pdb']


be.add_multiple_glycans(
    representation=protein_representation, assembly_name=name, 
    glycan_type=be.NAME_GLYCAN_COMPLEX,
    protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
    indices=[53, 90, 103, 322, 432, 690])

be.add_multiple_glycans(
    representation=protein_representation, assembly_name=name,
    glycan_type=be.NAME_GLYCAN_HYBRID,
    protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
    indices=[546])

indices = [[164, Quaternion(0.707, 0.0, 0.707, 0.0)],
           [739, Quaternion(0.707, 0.0, 0.707, 0.0)]]
for index in indices:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name, 
        source=o_glycan_paths[0],
        protein_name=name + '_' + be.NAME_RECEPTOR, 
        representation=protein_representation,
        chain_ids=[2, 4], site_indices=[index[0]], 
        rotation=index[1])
    be.add_sugars(o_glycan)

### Environment

In [ ]:
volume_position = Vector3(0.0, scene_size.y / 2.0 - 80.0, 0.0)

#### Glucose

In [ ]:
nglview.show_file(glucose_path)

In [ ]:
glucose = Protein(
    sources=[glucose_path], load_non_polymer_chemicals=True, 
    occurences=nb_glucoses)
volume = Volume(
    name=be.NAME_GLUCOSE, size=scene_size, protein=glucose)
status = be.add_volume(
    volume=volume, 
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=volume_position)

#### Lactoferrin

In [ ]:
nglview.show_file(lactoferrin_path)

In [ ]:
lactoferrin = Protein(
    sources=[lactoferrin_path], load_non_polymer_chemicals=True,
    occurences=nb_lactoferrins)
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN, size=scene_size, protein=lactoferrin)
status = be.add_volume(
    volume=lactoferrins_volume,
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=volume_position)

#### Defensins

In [ ]:
nglview.show_file(defensin_path)

In [ ]:
defensin = Protein(
    sources=[defensin_path], load_non_polymer_chemicals=True, 
    occurences=nb_defensins)
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, size=scene_size, protein=defensin)
status = be.add_volume(
    volume=defensins_volume, 
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=volume_position)

#### Surfactant-D

In [ ]:
def add_surfactant_d(name, position, random_seed):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, random_seed=random_seed)

add_surfactant_d(
    name='Surfactant-D 1',
    position=Vector3(2.0, 18.0, 102.0),
    random_seed=1)

#### Surfactant A

In [ ]:
def add_surfactant_a(name, position, random_seed):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, 
        representation=protein_representation,
        atom_radius_multiplier=protein_radius_multiplier,
        position=position, random_seed=random_seed)

add_surfactant_a(
    name='Surfactant-A 1',
    position=Vector3(-100.0, 0.0, 0.0),
    random_seed=2)

## Materials

In [ ]:
be.apply_default_color_scheme(
    specular_exponent=5, glossiness=0.1,
    user_parameter=0.01,
    shading_mode=be.SHADING_MODE_PERLIN)

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)